In [55]:
import re
from itertools import combinations, combinations_with_replacement, permutations, product 
from operator import itemgetter, add, mul, sub, truediv, pow
from sympy import sympify, symbols, exp, sin, cos, ln, asin, acos
from sympy.physics.units.quantities import Quantity
x, k = symbols('x k')

variables = [x]
constants = [k]

In [62]:
all_nodes = {
    'x': {
        'children': 0,
        'op': x
    },
    'k': {
        'children': 0,
        'op': k # Konstante
    },
    'exp': {
        'children': 1,
        'op': exp
    },
    'ln': {
        'children': 1,
        'op': ln
    },
    'sin': {
        'children': 1,
        'op': sin
    },
    'cos': {
        'children': 1,
        'op': cos
    },
    # 'asin': {
    #     'children': 1,
    #     'op': asin
    # },
    # 'acos': {
    #     'children': 1,
    #     'op': acos
    # },
    '+': {
        'children': 2,
        'op': add,
        'commutative': True,
        'no_duplicates': True # x+x = 2*x = c*x, so in the end a duplicate
    },
    '-': {
        'children': 2,
        'op': sub,
        'no_duplicates': True,
        'filter': lambda left, right: not right in constants
    },
    '*': {
        'children': 2,
        'op': mul,
        'commutative': True
    },
    '/': {
        'children': 2,
        'op': truediv,
        'no_duplicates': True
    },
    # '**': {
    #     'children': 2,
    #     'op': pow,
    #     'commutative': False
    # }
}

In [33]:
def combinatoric_iterator(operation):
    if 'commutative' in operation and 'no_duplicates' in operation:
        return lambda p: combinations(p, 2)
    elif 'commutative' in operation:
        return lambda p: combinations_with_replacement(p, 2)
    elif 'no_duplicates' in operation:
        return lambda p: permutations(p, 2)
    return lambda p: product(p, repeat=2)

### Komplexität
$k_1$: Anzahl nicht-kommutativer Operationen mit Duplikaten ($x^y$)

$k_2$: Anzahl nicht-kommutativer Operationen ohne Duplikate ($x-y$, $x/y$)

$k_3$: Anzahl kommutativer Operationen mit Duplikaten ($x\cdot y$)

$k_4$: Anzahl kommutativer Operationen ohne Duplikate ($x+y$)

$k_5$: Anzahl Funktionen ($sin$, $cos$, $exp$ etc.)

$n$: Anzahl Ausdrücke

$$f_k(n)=k_1n^2+k_2\frac{n!}{(n-2)!}+k_3\frac{(n+1)!}{2(n-1)!}+k_4\frac{n!}{2(n-2)!}+(k_5+1)n$$

Gesamtmenge aller Ausdrücke mit Tiefe $l$:

$$f^l_k(n)$$

In [61]:
def generate_expressions(sub_expressions):
    for properties in all_nodes.values():
        children, operation = itemgetter('children', 'op')(properties)
        if children == 2:
            iterator = combinatoric_iterator(properties)(sub_expressions)
            op_filter = properties.get('filter', None)
            for left, right in iterator:
                left, right = sympify(left), sympify(right)
                if not op_filter or op_filter(left, right):
                    yield operation(left, right)
        elif children == 1:
            for expr in sub_expressions:
                expr = sympify(expr)
                if expr != k:
                    yield operation(expr)

In [58]:
def save_expressions(depth):
    uniques = set([x, k])
    if depth > 1:
        with open(f'uniques_ext_depth{depth - 1}_red2.csv', 'r') as file:
            for line in file:
                uniques.add(line.strip())

    with open(f'expressions_ext_depth{depth}_red2.csv', 'w') as file:
        for expr in generate_expressions(uniques):
            expr_str = str(expr)
            if re.search(r'x(?!p)', expr_str):
                file.write(expr_str)
                file.write('\n')
        for expr in uniques:
            file.write(str(expr))
            file.write('\n')

In [57]:
def cleanup_expressions(depth, max_constants=None):
    uniques = set()
    re_mul_int = re.compile('(?<!\*)\*?\d\*(x|k)')
    def replace_mul_int(match):
        if match.group(1) == 'x':
            return 'k*x'
        return 'k'
    
    with open(f'expressions_ext_depth{depth}_red2.csv', 'r') as file:
        for line in file:
            line = re.sub(re_mul_int, replace_mul_int, line)
            line = line.replace('k**2', 'k')
            if max_constants and line.count('k') > max_constants:
                pass
            else:
                uniques.add(line)
    with open(f'uniques_ext_depth{depth}_red2.csv', 'w') as file:
        for line in uniques:
            file.write(line)

In [63]:
depth = 2
save_expressions(depth)
cleanup_expressions(depth)

In [10]:
cleanup_expressions(3, max_constants=2)